In [4]:
import os
import sys
import glob
import gzip
import pickle
import datetime

import numpy as np
import pandas as pd
import scipy 
from scipy import signal

# graph
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# signal prepocessing
import biosppy
from biosppy.signals import ecg

import pyhrv
import pyhrv.tools as tools
import pyhrv.time_domain as td

# sklean
from sklearn.ensemble import IsolationForest

#pyod isolation forest
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

In [ ]:
sk_isft = IsolationForest(max_samples=100, random_state=0)
sk_isft.fit(X_train)
'''
    X{array-like, sparse matrix} of shape (n_samples, n_features)

        The input samples. Use dtype=np.float32 for maximum efficiency. Sparse matrices are also supported, use sparse csc_matrix for maximum efficiency.
    yIgnored

        Not used, present for API consistency by convention.
    sample_weightarray-like of shape (n_samples,), default=None

        Sample weights. If None, then samples are equally weighted.
'''


In [ ]:
disp = DecisionBoundaryDisplay.from_estimator(
    sk_isft,
    X,
    response_method="decision_function",
    alpha=0.5,
)
disp.ax_.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor="k")
disp.ax_.set_title("Path length decision boundary \nof IsolationForest")
plt.axis("square")
plt.legend(handles=handles, labels=["outliers", "inliers"], title="true class")
plt.colorbar(disp.ax_.collections[1])
plt.show()

In [ ]:
contamination = 0.05 # percentage of outliers
n_train =        # number of training points
n_test =         # number of testing points
n_features =       # number of features
X_train, X_test, y_train, y_test = generate_data(
    n_train = n_train, 
    n_test = n_test, 
    n_features = n_features, 
    contamination = contamination, 
    random_state = 42)

X_train_pd = pd.DataFrame(X_train)
X_train_pd.head()

# Plot
plt.scatter(X_train_pd[0], X_train_pd[1], c=y_train, alpha=0.8)
plt.title('Scatter plot')
plt.xlabel('x0')
plt.ylabel('x1')
plt.show()

In [ ]:
po_isft = IForest(contamination=0.05, max_samples=40, behaviour='new') 
po_isft.fit(X_train)

# Training data
y_train_scores = isft.decision_function(X_train)
y_train_pred = isft.predict(X_train)

# Test data
y_test_scores = isft.decision_function(X_test)
y_test_pred = isft.predict(X_test) # outlier labels (0 or 1)

# Threshold for the defined comtanimation rate
print("The threshold for the defined contamination rate:" , isft.threshold_)

def count_stat(vector):
    # Because it is '0' and '1', we can run a count statistic. 
    unique, counts = np.unique(vector, return_counts=True)
    return dict(zip(unique, counts))

print("The training data:", count_stat(y_train_pred))
print("The training data:", count_stat(y_test_pred))

In [ ]:
print(po_isft.get_params())
po_isft_vi = isft.feature_importances_
print(po_isft_vi)

In [ ]:
for_plot = pd.DataFrame({'x_axis':X_train_pd.columns,
              'y_axis':isft_vi}).sort_values(by='y_axis',ascending=True)
for_plot['y_axis'].plot.barh()

In [ ]:
plt.hist(y_train_scores, bins='auto') # arguments are passed to np.histogram
plt.title("Outlier score")
plt.show()

In [ ]:
threshold = po_isft.threshold_ # Or other value from the above histogram

def descriptive_stat_threshold(df,pred_score, threshold):
    # Let's see how many '0's and '1's.
    df = pd.DataFrame(df)
    df['Anomaly_Score'] = pred_score
    df['Group'] = np.where(df['Anomaly_Score']< threshold, 'Normal', 'Outlier')

    # Now let's show the summary statistics:
    cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score':'Count'})
    cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
    stat = df.groupby('Group').mean().round(2).reset_index() # The avg.
    stat = cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together
    return (stat)

descriptive_stat_threshold(X_train,y_train_scores, threshold)